In [1]:
#! pip install -U langchain-google-genai langchain langchain-community langchain-experimental langchain-openai langchain-pinecone langchain-text-splitters -q --user

# Langchain
https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html
https://python.langchain.com/v0.2/docs/how_to/chatbots_retrieval/#document-chains
### create_retrieval_chain
https://python.langchain.com/v0.1/docs/use_cases/chatbots/retrieval/
### Chroma
https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/
### Pdf loader
https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/
### Google Genai
https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/

# Vertexai
https://cloud.google.com/vertex-ai/docs/general/custom-service-account

# GeminiAI
https://ai.google.dev/gemini-api/docs/api-key

# Chroma
https://docs.trychroma.com/integrations/google-gemini
https://docs.trychroma.com/guides/embeddings

In [1]:
import os, tempfile
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
import uuid


In [2]:
config = dotenv_values("keys/.env")
with open("keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

vertex_credentials = service_account.Credentials.from_service_account_info(info)
vertexai.init(
    project=config["PROJECT"],
    location=config["REGION"],
    credentials=vertex_credentials,
)
google_api_key = config["GEMINI-API-KEY"]
os.environ["GEMINI_API_KEY"] = google_api_key

In [3]:
client = chromadb.PersistentClient(path="./chroma_db")

In [4]:
embeddings_retriever = GoogleGenerativeAIEmbeddings(
                        model="models/embedding-001",
                        credentials=vertex_credentials,
                        google_api_key=google_api_key,
                    )

google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=google_api_key)

In [5]:
mypath = "./docs"
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [18]:
onlyfiles

['Baremo 2015.pdf',
 'BAREMO_AMA_BOE_RD_1971-1999.pdf',
 'BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf',
 'Clasificación de intervenciòn quirúrgica por nomenclator.pdf',
 'enfermedades profesionales de los agricultores.pdf',
 'Guia de Valoración Profesional.pdf',
 'GUIA_DE_VALORACION_DE_INCAPACIDAD_LABORAL_PARA_AP.pdf',
 'ley_enjuiciamiento_civil.pdf',
 'Proyecto de Ley Valoracion daÃ±os.pdf',
 'Tabla combinada de valoración de minusvalia.pdf',
 'Tablas_indemnizatorias_Baremo_2024.pdf',
 'Valoración del INSS de enfermedades para incapacidades.pdf']

In [6]:
def load_file(path):
    # load pdf file and transform into Langchain Documents
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    return pages

def get_docs_to_add_vectorstore(pages, file, google_ef):
    # get components to add to Chroma
    documents = []
    ids= []
    metadatas= []
    embeddings = []

    for page in pages:
        emb = google_ef([page.page_content])
        embeddings.append(emb[0])
        metadatas.append({"page": page.metadata.get("page"), "filename":file})
        ids.append(uuid.uuid1().hex)
        documents.append(page.page_content)
    return  documents,  ids, metadatas, embeddings 

In [7]:
collection = client.get_or_create_collection(name="forensic", embedding_function=google_ef)

In [8]:
# add documents to Chroma
for file in onlyfiles:
    path = os.path.join("docs", file)
    pages = load_file(path)
    print(f" Loaded file {file} with {len(pages)} ")
    documents,  ids, metadatas, embeddings = get_docs_to_add_vectorstore(pages, file, google_ef)
    # add documents to collection
    collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)
    print(collection.count())

 Loaded file Baremo 2015.pdf with 673 
673
 Loaded file BAREMO_AMA_BOE_RD_1971-1999.pdf with 139 
812
 Loaded file BAREMO_PARA_LA_CALIFICACION_DEL_GRADO_DE_MINUSVALIA_Anexo_I_RD_1971-1999.pdf with 283 
1095
 Loaded file Clasificación de intervenciòn quirúrgica por nomenclator.pdf with 254 
1349
 Loaded file enfermedades profesionales de los agricultores.pdf with 36 
1385
 Loaded file Guia de Valoración Profesional.pdf with 1211 
2596
 Loaded file GUIA_DE_VALORACION_DE_INCAPACIDAD_LABORAL_PARA_AP.pdf with 346 
2942
 Loaded file ley_enjuiciamiento_civil.pdf with 360 
3302
 Loaded file Proyecto de Ley Valoracion daÃ±os.pdf with 47 
3349
 Loaded file Tabla combinada de valoración de minusvalia.pdf with 5 
3354
 Loaded file Tablas_indemnizatorias_Baremo_2024.pdf with 42 
3396
 Loaded file Valoración del INSS de enfermedades para incapacidades.pdf with 411 
3807


In [9]:
# get vectorstore from Presistem directory

In [10]:
vectorstore = Chroma(persist_directory="./chroma_db/", embedding_function=embeddings_retriever, collection_name= "forensic")

In [11]:
retriever = vectorstore.as_retriever(
                        search_kwargs={"k": 5}
                    )

In [12]:
llm = ChatGoogleGenerativeAI(
                    model="gemini-1.5-pro-001", credentials=vertex_credentials
                )

In [13]:
template = """
                You are a helpful AI assistant. Answer based on the context provided. 
                context: {context}
                input: {input}
                answer:
                """
prompt = PromptTemplate.from_template(template)

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(
    retriever, combine_docs_chain
)
                


In [19]:
query="enumera las cinco categorías o clases de discapacidad"

In [35]:
response = retrieval_chain.invoke({"input": query})

In [36]:
response.keys()

dict_keys(['input', 'context', 'answer'])

In [37]:
print(response['answer'])

Las cinco categorías o clases de discapacidad, ordenadas de menor a mayor porcentaje, según la importancia de la deficiencia y el grado de discapacidad que origina son:

1. **CLASE I (0%):** Deficiencias permanentes diagnosticadas y tratadas, demostradas con parámetros objetivos, pero que no producen discapacidad. 
2. **CLASE II (1 - 24%):** Deficiencias permanentes que originan una discapacidad leve.
3. **CLASE III (25 - 49%):** Deficiencias permanentes que originan una discapacidad moderada.
4. **CLASE IV (50 - 70%):** Deficiencias permanentes que producen una discapacidad grave.
5. **CLASE V (75%):** Deficiencias permanentes severas que originan una discapacidad muy grave, suponiendo la dependencia de otras personas para realizar las actividades más esenciales de la vida diaria. 



In [38]:
len(response['context'])

5

In [39]:
print(response['context'])

[Document(page_content='NORMAS GENERALES\n25 Determinación del porcentaje de discapacidad\nTanto los grados de discapacidad como las actividades de la vida diaria descritos constitu-yen patrones de referencia para la asignación del porcentaje de discapacidad. Este porcen-taje se determinará de acuerdo con los criterios y clases que se especifican en cada uno delos capítulos.\nCon carácter general se establecen cinco categorías o clases, ordenadas de menor a\nmayor porcentaje, según la importancia de la deficiencia y el grado de discapacidad queorigina.\nEstas cinco clases se definen de la forma siguiente:CLASE I\nSe encuadran en esta clase todas las deficiencias permanentes que han sido diagnosticadas,tratadas adecuadamente, demostradas mediante parámetros objetivos (datos analíticos,radiográficos, etc., que se especifican dentro de cada aparato o sistema), pero que no pro-ducen discapacidad.\nLa calificación de esta clase es 0 %.CLASE II\nIncluye las deficiencias permanentes que, cump